<a href="https://colab.research.google.com/github/owosade/Explainable-AI---Final/blob/main/Saliency_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install torch torchvision captum

# Import libraries
import torch
from torchvision import models, transforms
from captum.attr import LayerGradCam

In [ ]:
#Importing fastai
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
#Importing all objects and functions from the fastai module
from fastbook import *
from fastai.vision.all import *
# Import necessary libraries
import torch
import torch.nn as nn
from fastai.vision.all import *
from torchvision import models, transforms
from captum.attr import LayerGradCam



#creating an object named "Path"
path = Path('/content/gdrive/MyDrive/Dataset')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.0 MB/s eta 0:00:00
Mounted at /content/gdrive


In [ ]:
#test_items = get_image_files(path/"Test")
files= get_image_files(path)

#data modification
augumentations = [RandomResizedCropGPU(size=224, min_scale=0.75), Rotate(), Zoom()]

#data block creation
data_block = DataBlock(blocks=(ImageBlock(cls=PILImage), CategoryBlock),
                   splitter=GrandparentSplitter(train_name='Train', valid_name='Test'),
                   get_y=parent_label,
                   item_tfms=Resize(512, method="squish"),
                   batch_tfms=augumentations,
                   )

load_data_test = data_block.dataloaders(files)

model = nn.Sequential(create_body(xresnet34(), pretrained=True),create_head(nf=2048, n_out=2))

#Loading the saved model
learn = cnn_learner(load_data_test, resnet34, metrics=accuracy).to_fp16()
model = learn.load('/content/gdrive/MyDrive/Dataset/Final_project4')
for name, module in model.named_modules():
    print(name)

for name, child in model.named_children():
    print(name, child)

# Find and print the last layer
last_layer_name, last_layer = list(model.named_children())[-1]
print("Last Layer:", last_layer_name, last_layer)

/usr/local/lib/python3.10/dist-packages/fastai/vision/learner.py:301: UserWarning: `cnn_learner` has been renamed to `vision_learner` -- please update your code
  warn("`cnn_learner` has been renamed to `vision_learner` -- please update your code")
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:03<00:00, 22.1MB/s]



0
0.0
0.1
0.2
0.3
0.4
0.4.0
0.4.0.conv1
0.4.0.bn1
0.4.0.relu
0.4.0.conv2
0.4.0.bn2
0.4.1
0.4.1.conv1
0.4.1.bn1
0.4.1.relu
0.4.1.conv2
0.4.1.bn2
0.4.2
0.4.2.conv1
0.4.2.bn1
0.4.2.relu
0.4.2.conv2
0.4.2.bn2
0.5
0.5.0
0.5.0.conv1
0.5.0.bn1
0.5.0.relu
0.5.0.conv2
0.5.0.bn2
0.5.0.downsample
0.5.0.downsample.0
0.5.0.downsample.1
0.5.1
0.5.1.conv1
0.5.1.bn1
0.5.1.relu
0.5.1.conv2
0.5.1.bn2
0.5.2
0.5.2.conv1
0.5.2.bn1
0.5.2.relu
0.5.2.conv2
0.5.2.bn2
0.5.3
0.5.3.conv1
0.5.3.bn1
0.5.3.relu
0.5.3.conv2
0.5.3.bn2
0.6
0.6.0
0.6.0.conv1
0.6.0.bn1
0.6.0.relu
0.6.0.conv2
0.6.0.bn2
0.6.0.downsample
0.6.0.downsample.0
0.6.0.downsample.1
0.6.1
0.6.1.conv1
0.6.1.bn1
0.6.1.relu
0.6.1.conv2
0.6.1.bn2
0.6.2
0.6.2.conv1
0.6.2.bn1
0.6.2.relu
0.6.2.conv2
0.6.2.bn2
0.6.3
0.6.3.conv1
0.6.3.bn1
0.6.3.relu
0.6.3.conv2
0.6.3.bn2
0.6.4
0.6.4.conv1
0.6.4.bn1
0.6.4.relu
0.6.4.conv2
0.6.4.bn2
0.6.5
0.6.5.conv1
0.6.5.bn1
0.6.5.relu
0.6.5.conv2
0.6.5.bn2
0.7
0.7.0
0.7.0.conv1
0.7.0.bn1
0.7.0.relu
0.7.0.conv2
0.7.0.bn2
0

In [ ]:
# Function to recursively get the last layer name
def get_last_layer_name(module, prefix=''):
    # Check if the module has sub-modules
    if len(list(module.children())) > 0:
        # Recursively call the function on the sub-modules
        return get_last_layer_name(list(module.children())[-1], prefix + f"{module.__class__.__name__}.")
    else:
        # Return the name of the current module
        return prefix + module.__class__.__name__

# Get the last layer name using the function
last_layer_name = get_last_layer_name(model)
print("Last Layer Name:", last_layer_name)

Last Layer Name: Learner.Sequential.Linear


In [ ]:
layer_name = 'Learner.Sequential.Linear'
layer = getattr(model, layer_name, None)
if layer is not None:
    # Register the forward hook on the chosen layer
    hook_handle = layer.register_forward_hook(forward_hook)

    # Perform a forward pass to trigger the hook
    input_data = torch.randn(1, 3, 224, 224)  # Example input data
    model(input_data)

    # Remove the hook when done
    hook_handle.remove()
else:
    print(f"Layer '{layer_name}' not found in the model.")

Layer 'Learner.Sequential.Linear' not found in the model.


In [ ]:

model.eval()
# Access the last layer in the model (assuming it's the head)
last_layer = "Learner.Sequential.Linear"
layer = getattr(model, last_layer, None)

# Create a LayerGradCam object
layer_grad_cam = LayerGradCam(learn.model, last_layer)


# Choose an image from your dataset for generating saliency map
image_path = '/content/gdrive/MyDrive/Dataset/Test/Kidney_stone/1.3.46.670589.33.1.63706830475347975400001.4676991400730475635.png'  # Replace with the actual path
image = Image.open(image_path).convert('RGB')  # Ensure it's RGB

# Preprocess the image
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_image = preprocess(image).unsqueeze(0)

# Generate the saliency map
saliency_map = layer_grad_cam.attribute(input_image)

# Visualize the original image and the saliency map
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# Original image
ax1.imshow(image)
ax1.set_title('Original Image')

# Saliency map
ax2.imshow(saliency_map.squeeze().numpy(), cmap='hot')
ax2.set_title('Saliency Map')

plt.show()


AttributeError: 'str' object has no attribute 'register_forward_hook'